In [2]:
import pandas as pd
import os
import tkinter as tk
from tkinter import filedialog

In [26]:
# Search for LIV Raw File
root = tk.Tk()
root.withdraw()
root.lift()
root.attributes("-topmost", True)
file_path = filedialog.askopenfilename(title="Select COD File", filetypes=(("csv files", "*.csv"),))
df = pd.read_csv(file_path, skiprows=19)
folder = os.path.dirname(file_path)
print("Directory : {}".format(folder))

Directory : C:/Users/762093/Documents/LIV/liv_raw/LIV_Raw_Files


HALO Decoder

In [27]:
sd = df
# print(sd.head(20))

    TOUCHDOWN  STX_WAFER_X_UM  STX_WAFER_Y_UM  CUBE  CHANNEL_NUM       Vf1  \
0           1          -32339          -48416    64            1  1.366367   
1           2          -32083          -48449    64            1  1.366444   
2           3          -31826          -48482    64            1  1.366831   
3           4          -31569          -48416    64            1  1.366792   
4           5          -31313          -48449    64            1  1.367333   
5           6          -31056          -48482    64            1  1.367488   
6           7          -30799          -48416    64            1  1.367449   
7           8          -30543          -48449    64            1  4.999843   
8           9          -30286          -48482    64            1  1.367642   
9          10          -30029          -48416    64            1  4.999689   
10         11          -29773          -48449    64            1  5.000114   
11         12          -29516          -48482    64            1

In [6]:
cols = list(df.columns)

In [7]:
# Use list comprehension to filter values starting with "PD"
# The startswith() method returns True if a string starts with the specified prefix(string).
filtered_list = [item for item in cols if item.startswith("PD")]
filtered_list.remove("PD@35mA")

In [8]:
# Step 1: Extract numbers from the strings and convert them to integers
numbers = [int(item[2:]) for item in filtered_list]
# Step 2: Find the maximum number in the list
sweep_len = max(numbers)
# 5 starter columns + sweep length *2
pd_max = sweep_len * 2 + 5

In [9]:
# non_sweep_data = df.iloc[:,pd_max:]
df = df.iloc[:, :pd_max]
cols = list(df.columns)
cols[-1]

'PD70'

In [10]:
# Keep the first five columns the same, transpose the rest of the columns with the new col metric being the headers
# and value being the col values
df = df.melt(
    id_vars=["TOUCHDOWN", "STX_WAFER_X_UM", "STX_WAFER_Y_UM", "CUBE", "CHANNEL_NUM"],
    var_name="METRIC",
)

In [12]:
# Splitting the Metric column into two cols based on the first two characters
df["MEASUREMENT"] = df["METRIC"].str[:2]
df["CURRENT_MA"] = df["METRIC"].str[2:]
# No longer need metric col
df = df.drop(columns=["METRIC"])

KeyError: 'METRIC'

In [10]:
## Split by the measurement column
pd = df[df["MEASUREMENT"] == "PD"]
pd = pd.rename(columns={"value": "PD_MW"})
pd = pd.drop(columns=["MEASUREMENT"])
pd.head()

,TOUCHDOWN,STX_WAFER_X_UM,STX_WAFER_Y_UM,CUBE,CHANNEL_NUM,PD_MW,CURRENT_MA
106260,1,-32339,-48416,64,1,0.172598,1
106261,2,-32083,-48449,64,1,0.166555,1
106262,3,-31826,-48482,64,1,0.166555,1
106263,4,-31569,-48416,64,1,0.168066,1
106264,5,-31313,-48449,64,1,0.155979,1


In [11]:
vf = df[df["MEASUREMENT"] == "Vf"]
vf = vf.rename(columns={"value": "VF"})
vf = vf.drop(columns=["MEASUREMENT"])
vf.head()

,TOUCHDOWN,STX_WAFER_X_UM,STX_WAFER_Y_UM,CUBE,CHANNEL_NUM,VF,CURRENT_MA
0,1,-32339,-48416,64,1,1.366367,1
1,2,-32083,-48449,64,1,1.366444,1
2,3,-31826,-48482,64,1,1.366831,1
3,4,-31569,-48416,64,1,1.366792,1
4,5,-31313,-48449,64,1,1.367333,1


In [12]:
df = vf.merge(pd)
df = df.iloc[:, [0, 1, 2, 4, 3, 6, 5, 7]]

In [13]:
# Change Current col to integer datatype
df["CURRENT_MA"] = df["CURRENT_MA"].astype("int64")
# Delete Existing Cube Col
df = df.drop(columns=["CUBE"])

In [14]:
df = df.merge(sd, on=["STX_WAFER_X_UM", "STX_WAFER_Y_UM"])

In [15]:
# Obtain the first derivative of PD
df["DL_DI"] = df["PD_MW"].diff() / df["CURRENT_MA"].diff()
df.head()

,TOUCHDOWN,STX_WAFER_X_UM,STX_WAFER_Y_UM,CHANNEL_NUM,CURRENT_MA,VF,PD_MW,CUBE,TE_LABEL,DL_DI
0,1,-32339,-48416,1,1,1.366367,0.172598,64,64N6V,NaN
1,1,-32339,-48416,1,2,1.430014,0.174109,64,64N6V,0.001511
2,1,-32339,-48416,1,3,1.463484,0.172598,64,64N6V,-0.001511
3,1,-32339,-48416,1,4,1.487101,0.171087,64,64N6V,-0.001511
4,1,-32339,-48416,1,5,1.506158,0.171087,64,64N6V,0.000000


#### File Naming and Saving

In [16]:
### ----------------- File Naming and Saving -----------------------------

# Create new file name with processed suffix
file_path = os.path.basename(file_path).split(".")[0]
file_path = file_path.rsplit("_", 1)[0] + "_sweeps.csv"
print("File name : {}".format(file_path))
## combine the diretory and new filename
new_path = os.path.join(folder, file_path)
print("Full Path Directory : {}".format(new_path))

File name : LIV_53_QCHW2_DNS-LIVTKCOD_COD70-DNS_sweeps.csv
Full Path Directory : C:/Users/534399/OneDrive - Seagate Technology/Vadan/LIV Raw Files\LIV_53_QCHW2_DNS-LIVTKCOD_COD70-DNS_sweeps.csv


In [17]:
mach = file_path[:6]
lot_id = file_path[7:12]
df["WAFER_ID"] = lot_id
df["MACH_ID"] = mach

In [18]:
# Write to the new path location, header first then append with data
df.to_csv(new_path, index=False)